In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
# Step 1 - load data
dataframe = pd.read_csv('live_demo.csv')
dataframe = dataframe.drop(['index', 'price', 'sq_price'], axis=1)
dataframe = dataframe[0:10]
dataframe

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0
5,1985.0,4.0
6,1534.0,3.0
7,1427.0,3.0
8,1380.0,3.0
9,1494.0,3.0


In [4]:
# Step 2 - add labels
# 1 is good and 0 is bad
dataframe.loc[:, ('y1')] = [1, 1, 1, 0, 0, 1, 0, 1, 1, 1]
# y2 is a negation of y1
dataframe.loc[:, ('y2')] = (dataframe['y1'] == 0).astype(int)
dataframe

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,0,1
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


In [5]:
# Step 3 - prepare data for tensorflow
# tensors are a generic version of vectors and matrics
# vector - list of numbers (1D tensor)
# matrix - list of list of numbers (2D tensor)

# convert features to input tensor
inputX = dataframe.loc[:, ['area', 'bathrooms']].as_matrix()

# convert labels to input tensors
inputY = dataframe.loc[:, ['y1', 'y2']].as_matrix()

In [6]:
# Step 4 - write out our hyperparameters
# how fast to convergence
learning_rate = 0.000001
training_epochs = 2000
display_step = 50
n_samples = inputY.size

In [12]:
# Step 5 - create our computation graph / neural network

# for feature input tensors
# None means any numbers of examples
# 2 because we have two features
# placeholders are gateways for data into our computation graph
x = tf.placeholder(tf.float32, [None, 2])

# create weights
# 2 x 2 float matrix that we will keep updating through the
# training process
# variables in tf hold and update parameters
# in memory buffers containing tensors
W = tf.Variable(tf.zeros([2, 2]))

# add biases (similar to b in y = mx + b)
b = tf.Variable(tf.zeros([2]))

# multiply weights by inputs, first calculation
# weights are how we govern how data flows in our computation graph
# multiply input by weights and add biases
y_values = tf.add(tf.matmul(x, W), b)

# apply softmax to the value we just created
# softmax is our activation function
y = tf.nn.softmax(y_values)

# feed in a matrix of labels
y_ = tf.placeholder(tf.float32, [None, 2])

In [29]:
# Step 6 - perform training
# create our cost function, mean squared error
# reduce sum computes the sum of elements across dimentions of a tensor
cost = tf.reduce_sum(tf.pow(y_ - y, 2)) / (2 * n_samples)

# gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# initialize variables and tensorflow session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# training loop
for i in range(training_epochs):
    sess.run(optimizer, feed_dict = {x: inputX, y_: inputY})
    # write out logs of training
    if (i) % display_step == 0:
        cc = sess.run(cost, feed_dict = {x: inputX, y_: inputY})
        print('training step:', '%04d' % (i), 'cost=', '{:.9f}'.format(cc))

print('Optimization finished!')
training_cost = sess.run(cost, feed_dict = {x: inputX, y_: inputY})
print('Training cost = {0}, W = {1}, b = {2}'.format(training_cost, sess.run(W), sess.run(b)))


training step: 0000 cost= 0.114958666
training step: 0050 cost= 0.109539941
training step: 0100 cost= 0.109539881
training step: 0150 cost= 0.109539807
training step: 0200 cost= 0.109539732
training step: 0250 cost= 0.109539673
training step: 0300 cost= 0.109539606
training step: 0350 cost= 0.109539531
training step: 0400 cost= 0.109539464
training step: 0450 cost= 0.109539405
training step: 0500 cost= 0.109539330
training step: 0550 cost= 0.109539248
training step: 0600 cost= 0.109539196
training step: 0650 cost= 0.109539129
training step: 0700 cost= 0.109539054
training step: 0750 cost= 0.109538987
training step: 0800 cost= 0.109538913
training step: 0850 cost= 0.109538853
training step: 0900 cost= 0.109538779
training step: 0950 cost= 0.109538712
training step: 1000 cost= 0.109538652
training step: 1050 cost= 0.109538577
training step: 1100 cost= 0.109538510
training step: 1150 cost= 0.109538436
training step: 1200 cost= 0.109538376
training step: 1250 cost= 0.109538302
training ste

In [35]:
sess.run(y, feed_dict = {x: inputX})

array([[ 0.71125221,  0.28874779],
       [ 0.66498977,  0.33501023],
       [ 0.73657656,  0.26342347],
       [ 0.64718789,  0.35281211],
       [ 0.78335613,  0.2166439 ],
       [ 0.70069474,  0.29930523],
       [ 0.65866327,  0.34133676],
       [ 0.64828628,  0.35171372],
       [ 0.64368278,  0.35631716],
       [ 0.65480113,  0.3451989 ]], dtype=float32)

Reference: https://github.com/llSourcell/How_to_use_Tensorflow_for_classification-LIVE/blob/master/demo.ipynb